## Dataset

## 爬虫 2017年的所有的新闻

In [1]:
!ls article_9k.txt

article_9k.txt


In [2]:
!less article_9k.txt

此外自本周6月12日起除小米手机6等15款机型外其余机型已暂停更新发布含开发版体验版内测稳定版暂不受影响以确保工程师可以集中全部精力进行系统优化工作有人猜测这也是将精力主要用到MIUI9的研发之中MIUI8去年5月发布距今已有一年有余也是时候更新换代了当然
关于MIUI9的确切信息我们还是等待官方消息
骁龙835作为唯一通过Windows10桌面平台认证的ARM处理器高通强调不会因为只考虑性能而
去屏蔽掉小核心相反他们正联手微软找到一种适合桌面平台的兼顾性能和功耗的完美方案报道称微软已经拿到了一些新的源码以便Windows10更好地理解biglittle架构资料显示骁龙835作为一款集成了CPUGPU基带蓝牙WiFi的SoC比传统的Wintel方案可以节省至少30的PCB空间
按计划今年Q4华硕惠普联想将首发骁龙835Win10电脑预计均是二合一形态的产品当然高通骁龙只是个开始未来也许还能见到三星Exynos联发科华为麒麟小米澎湃等进入Windows10桌面
平台
此前的一加3T搭载的是3400mAh电池DashCharge快充规格为5V4A至于电池缩水可能与刘作虎
所说一加手机5要做市面最轻薄大屏旗舰的设定有关按照目前掌握的资料一加手机5拥有55寸1080P三星AMOLED显示屏6G8GBRAM64GB128GBROM双1600万摄像头备货量惊喜根据京东泄露的
信息一加5起售价是xx99元应该是在279928992999中的某个
这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车新华社记者张立云
摄
向的逆行辅道上发现该女子女子身上一丝不挂地逆车流而行时走时停时坐时躺险象环生刘青:行走在南坪快速上期间还起了轻生年头一辅警发现后赶紧为其披上黄衣并一路劝说她那么事发时到底都发生了些什么呢南都记者带您一起还原现场南都记者在龙岗大队坂田中队见到了辅警刘青发现女生的辅警一位外表高大帅气说话略带些腼腆的90后青年刘青介绍6月16日早上7时36分他正在环城南路附近值勤接到中队关于一位女子裸身进入机动车可能有危险的警情随后骑着小铁骑开始沿路寻找大概花了十多分钟在南坪大道坂田出口往龙岗方article_9k.txt

In [3]:
chinese_dataset = 'article_9k.txt'

In [4]:
CHINESE_CHARATERS = open(chinese_dataset).read()

In [5]:
CHINESE_CHARATERS[:10]

'此外自本周6月12日'

In [6]:
len(CHINESE_CHARATERS)

33425826

In [7]:
import pinyin

In [8]:
pinyin.get('你好', format="strip", delimiter=" ")

'ni hao'

In [9]:
def chinese_to_pinyin(character):
    return pinyin.get(character, format="strip", delimiter=" ")

In [16]:
chinese_to_pinyin('你好吗？')

'ni hao ma ？'

In [14]:
CHINESE_PINYIN_CORPYS = chinese_to_pinyin(CHINESE_CHARATERS)

In [15]:
CHINESE_PINYIN_CORPYS[:100]

'ci wai zi ben zhou 6 yue 1 2 ri qi chu xiao mi shou ji 6 deng 1 5 kuan ji xing wai qi yu ji xing yi '

In [18]:
import re # regular expression

In [20]:
def tokens(text):
    "List all the pinyin characrters"
    return re.findall('[a-z]+', text.lower()) 

In [19]:
some_strange_str = "1st You are a very very .... strange man!!! "

In [21]:
tokens(some_strange_str)

['st', 'you', 'are', 'a', 'very', 'very', 'strange', 'man']

In [22]:
TOKENS = tokens(CHINESE_PINYIN_CORPYS)

## 没有一个叫做“常量”的东西，const

In [30]:
from collections import Counter

In [35]:
PINYIN_COUNT = Counter(TOKENS)

In [38]:
PINYIN_COUNT.most_common(20)

[('shi', 860634),
 ('de', 809887),
 ('yi', 682478),
 ('ji', 645276),
 ('guo', 430042),
 ('zhong', 409418),
 ('zhi', 398612),
 ('xin', 359619),
 ('li', 355441),
 ('zai', 334106),
 ('wei', 326301),
 ('hua', 304941),
 ('yu', 302949),
 ('she', 293312),
 ('he', 285689),
 ('bu', 281533),
 ('ri', 278379),
 ('jin', 278141),
 ('you', 277726),
 ('xian', 269047)]

## Lanauge Model

+ input: string
+ output:  0 ~ 1 

$$ String -> word_0 word_1 word_2 $$

$$ Pr(string) = Pr(w_0 w_1 w_2 w_3)         $$
$$ 2-gram Pr(w_0 w_1 w_2 w_3)  = \frac{Pr(w_0 w_1)}{Pr(w_1)} \cdot \frac{Pr(w_1 w_2)}{Pr(w_2)} \cdot \frac{Pr(w_2 w_3)}{Pr(w_3)} \cdot Pr(w_3) $$
$$ 1-gram Pr(w_0 w_1 w_2 w_3)  = Pr(w_0) \dots 
Pr(w_3) $$

In [73]:
def prob_one_word(word):
    total = sum(PINYIN_COUNT.values())
    if word in PINYIN_COUNT: 
        return PINYIN_COUNT[word] / total
    else:
        return min(PINYIN_COUNT.values()) / total # Out Of Vacabulary

In [47]:
prob_one_word('ni')

0.001348378327943515

In [61]:
def _1_gram_prob_words(words):
    return reduce(op.mul, [prob_one_word(w) for w in words])

In [78]:
_2_gram_words = [
    TOKENS[i] + TOKENS[i+1] for i in range(len(TOKENS)-1)
]

In [79]:
_2_gram_words[:10]

['ciwai',
 'waizi',
 'ziben',
 'benzhou',
 'zhouyue',
 'yueri',
 'riqi',
 'qichu',
 'chuxiao',
 'xiaomi']

In [80]:
_2_GRAM_COUNT = Counter(_2_gram_words)

In [81]:
def prob_2_word(word):
    total = sum(_2_GRAM_COUNT.values())
    if word in _2_GRAM_COUNT: 
        return _2_GRAM_COUNT[word] / total
    else:
        return min(_2_GRAM_COUNT.values()) / total

In [82]:
prob_2_word('wocai')

3.3894212198744405e-06

In [83]:
prob_2_word('woxiang')

2.8969958728360782e-05

In [84]:
prob_2_word('wolai')

7.354404533689823e-06

In [85]:
def _2_gram_model(words: str):
    
    words = words.split()
    
    prob = 1
    
    for i in range(len(words) - 1):
        word = words[i]
        next_word = words[i+1]
        
        _two_gram_prob = prob_2_word(word+next_word)
        _one_gram_prob = prob_one_word(next_word)
        
        prob *= _two_gram_prob / _one_gram_prob
    
    prob *= prob_one_word(words[-1])
    
    return prob

In [90]:
string1 = 'wo xiang shang qin hua da xue'
string2 = 'wo xiang shang qing hua da xue' # <- 
string3 = 'wo xiang shang qing hua da xiu'

In [87]:
_2_gram_model(string1)

1.7762036554131007e-16

In [88]:
_2_gram_model(string2)

2.3136988054823888e-15

In [89]:
_2_gram_model(string3)

2.648010077805309e-17

In [62]:
from functools import reduce

In [63]:
import operator as op

In [76]:
_1_gram_prob_words('zhe ge hennnn qi guai'.split())

5.281580104131215e-18

In [77]:
_1_gram_prob_words('zhe ge gennnng hennnn qi guai'.split())

2.4177790357182762e-24

In [65]:
_1_gram_prob_words('wo zai zuo ce shi'.split())

3.0883637741169432e-12

In [ ]:
_1_gram_prob_words('wo zai zuo ce shi'.split())

In [67]:
_1_gram_prob_words(string1.split())

1.126589655032671e-17

In [68]:
_1_gram_prob_words(string2.split())

5.980668075496288e-17

In [69]:
_1_gram_prob_words(string3.split())

1.6954478376525142e-17

## 截止目前 我们能判断 句子的概率了！

## 3： 修改、比、对

In [138]:
def edist0(word):
    return {word}

def edist1(word):
    'test'
    't^est, te^st, '
    split = get_splits(word)
    deletes = get_deletes(split)
    inserts = get_inserts(split)
    replace = get_replaces(split)
    transpose = get_transposes(split)
    
    return set(deletes + inserts + replace + transpose)

In [123]:
len(edist1('wo'))

130

In [124]:
def edist2(word):
    return {e2 for e1 in edist1(word) for e2 in edist1(e1)}            

In [127]:
len(edist2('wo'))

7154

In [129]:
def known(words):
    return {w for w in words if w in PINYIN_COUNT}

In [130]:
len(known(edist2('wo')))

146

In [109]:
def get_splits(word):
    "Return a list of all possible (first, rest) pairs that comprise pinyin."
    return [(word[:i], word[i:]) 
            for i in range(len(word)+1)]

In [93]:
get_split('chang')

[('', 'chang'),
 ('c', 'hang'),
 ('ch', 'ang'),
 ('cha', 'ng'),
 ('chan', 'g'),
 ('chang', '')]

In [94]:
alphabet = 'abcdefghijklmnopqrstuvwxyz'

In [114]:
def get_deletes(pairs):
    return [a+b[1:] for (a, b) in pairs if b]

def get_transposes(pairs):
    return [a+b[1]+b[0]+b[2:] for (a, b) in pairs if len(b) > 1]

def get_replaces(pairs):
    return [a+c+b[1:]  for (a, b) in pairs for c in alphabet if b]

def get_inserts(pairs):
    return [a+c+b for (a, b) in pairs for c in alphabet]

In [98]:
get_deletes(get_split('meng'))

['eng', 'mng', 'meg', 'men']

In [99]:
get_transposes(get_split('meng'))

['emng', 'mneg', 'megn']

In [106]:
len(get_inserts(get_split('meng')))

130

## First 我们能判断句子的概率了
## Second 我们能修改生成拼音了

In [139]:
def get_candidates(word):
    candidates = (known(edist0(word)) or 
                  known(edist1(word)) or 
                  known(edist2(word)) or 
                  [word])
    return candidates

In [169]:
def simple_correct(word):
    "Find the most possible pinyin based on edit distance."
    
    # Prefer edit distance 0, then 1, then 2; otherwise default to word itself.
    
    candidates = get_candidates(word)
    
    return max(candidates, key=PINYIN_COUNT.get) # return the most possible 

In [173]:
simple_correct('xue')

'xue'

In [174]:
simple_correct('xuue')

'xue'

In [175]:
def simple_correct_sequence(words):
    return ' '.join(map(simple_correct, words.split()))

In [176]:
simple_correct_sequence('zhe shi ge ce sho')

'zhe shi ge ce shi'

In [177]:
simple_correct_sequence('wo xiang shang da xuue')

'wo xiang shang da xue'

In [178]:
simple_correct('shii')

'shi'

In [179]:
simple_correct('yeu')

'yu'

In [180]:
simple_correct('yyt')

'yi'

In [181]:
simple_correct('kiang')

'xiang'

In [182]:
simple_correct('sho') # shi

'shi'

In [183]:
simple_correct_sequence('zhe shi ge ce sho')

'zhe shi ge ce shi'

In [184]:
simple_correct_sequence('wo xiang ssang xyu')

'wo xiang shang yu'

## 
+ 9k dataset 
+ 维基百科等等

## 改的更复杂一些，能够处理上下文相关！

In [191]:
def get_candidates_deep(word):
    candidates = (known(edist0(word)) |
                  known(edist1(word)) |
                  known(edist2(word)) |
                  set([word]))

    return candidates    

In [198]:
def correct_words_pair_2_gram(word_pair, topn=10):
    head, tail = word_pair.split()

    head_candidates = get_candidates_deep(head)
    tail_candidates = get_candidates_deep(tail)
    
    return sorted([(h, t) for h in head_candidates for t in tail_candidates], 
               key=lambda x:_2_gram_model(' '.join(x)), reverse=True)[:topn]

## todo: 咱们可以把现在处理2个单词的词组，改成处理句子的！

In [199]:
correct_words_pair_2_gram('ssang xyu') # 

[('xiang', 'mu'),
 ('chang', 'yi'),
 ('wang', 'qiu'),
 ('lang', 'pu'),
 ('jiang', 'su'),
 ('jiang', 'yu'),
 ('bang', 'zhu'),
 ('shang', 'wu'),
 ('shan', 'xi'),
 ('sheng', 'wu')]

In [204]:
correct_words_pair_2_gram('hu jian', topn=20) # 

[('er', 'xian'),
 ('guo', 'ji'),
 ('she', 'ji'),
 ('fa', 'zhan'),
 ('guo', 'jia'),
 ('shi', 'jie'),
 ('shi', 'jian'),
 ('she', 'xin'),
 ('yi', 'yuan'),
 ('ri', 'dian'),
 ('yi', 'ji'),
 ('mu', 'qian'),
 ('shi', 'ji'),
 ('zi', 'ji'),
 ('shi', 'xian'),
 ('qiu', 'yuan'),
 ('an', 'quan'),
 ('de', 'ji'),
 ('ke', 'ji'),
 ('yi', 'jing')]

## Language Model
## Edit Distance

## Token, Counter, RE, NLP